In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_face_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected faces to visualize.
    for idx in range(len(face_landmarks_list)):
        face_landmarks = face_landmarks_list[idx]

        # Draw the face landmarks.
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
          landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])

        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_tesselation_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_contours_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp.solutions.drawing_styles
              .get_default_face_mesh_iris_connections_style())

    return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
    # Extract the face blendshapes category names and scores.
    face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
    face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
    # The blendshapes are ordered in decreasing score value.
    face_blendshapes_ranks = range(len(face_blendshapes_names))

    fig, ax = plt.subplots(figsize=(12, 12))
    bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
    ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
    ax.invert_yaxis()

    # Label each bar with values
    for score, patch in zip(face_blendshapes_scores, bar.patches):
        plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

    ax.set_xlabel('Score')
    ax.set_title("Face Blendshapes")
    plt.tight_layout()
    plt.show()

In [5]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import scipy as scp

vid_width = 1280
vid_height = 720

vid = cv2.VideoCapture('./raw_training_data/1/1.mp4')
codec = cv2.VideoWriter_fourcc('M','J','P','G')
vid.set(6, codec)
vid.set(5, 30)
vid.set(3, vid_width)
vid.set(4, vid_height)
out = cv2.VideoWriter('./raw_training_data/1/output.avi',codec, 10, (vid_width,vid_height))

BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult

face_options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='face_landmarker.task'),
    running_mode=VisionRunningMode.VIDEO,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=False,
    num_faces=2,
    min_face_detection_confidence=0.5,
    min_face_presence_confidence=0.5,
    min_tracking_confidence=0.5)

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

with FaceLandmarker.create_from_options(face_options) as facelandmarker:
        while (vid.isOpened()):

            #Read frame
            ret, frame = vid.read()
            frame_timestamp_ms = round(time.time()*1000)
            
            #Detect faces with Haar cascade
            faces = face_classifier.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 1.1, 10, minSize=(80, 80))
            
            #Add old faces according to face tracker
            
            #Zoom faces
            for (x, y, w, h) in faces:
                lg_w = int( 1.4 * w)
                lg_h = int(16.  * w / 9)
                x = int(x - abs(lg_w - w)/2.0)
                y = int(y - abs(lg_h - h)/2.0)
                frame = frame[x:(x + lg_w),y:(y + lg_h),:]
                scaled_dim_lengths = np.int32(np.floor(np.divide(np.shape(frame)[0:2],1.25)))
                begin_idxs = np.int32(np.ceil(np.subtract(np.divide(np.shape(frame)[0:2],2),np.divide(np.shape(frame)[0:2],2))))
                end_idxs = np.add(begin_idxs,scaled_dim_lengths)
                width_idxs = np.arange(begin_idxs[0],end_idxs[0])
                height_idxs = np.arange(begin_idxs[1],end_idxs[1])
                cropped = frame[width_idxs, height_idxs]
                np.take(frame, np.arange(begin_idxs[0],end_idxs[0]))
                zoomed_frame_ch1 = scp.ndimage.zoom(frame[:,:,1], 1.25)
                begin_idxs = (np.shape(zoomed_frame)-[vid_width, vid_height])/2
                end_idxs = begin_idxs + [vid_width, vid_height]
                #frame = np.take(zoomed_image, np.arange(begin_idxs,end_idxs))

                #Detect face landmarks from each zoomed face.
                face_landmarker_result = facelandmarker.detect_for_video(mp.Image(image_format=mp.ImageFormat.SRGB, data=frame), frame_timestamp_ms)
            
            #Annotate frame with landmarks
            annotated_image = draw_face_landmarks_on_image(image.numpy_view(), face_landmarker_result)
            
            #Annotate frame with face bounding boxes
            for (x, y, w, h) in faces:
                lg_w = int( 1.4 * w)
                lg_h = int(16.  * w / 9)
                x = int(x - abs(lg_w - w)/2.0)
                y = int(y - abs(lg_h - h)/2.0)
                cv2.rectangle(annotated_image, (x, y), (x + lg_w, y + lg_h), (0, 255, 0), 4)

            #Display frame
            cv2.imshow("Face landmarks", annotated_image)
            out.write(annotated_image)

            if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
                vid.release()
                break

out.release()
cv2.destroyWindow("Face landmarks") 
cv2.waitKey(1)

W20230821 12:46:10.974512 841203 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.


-1